# 🏆 Progetto Finale: Analisi E-Commerce Avanzata

Questo progetto integra TUTTO ciò che abbiamo imparato:
- Merge su dataset multipli
- Filtri complessi
- Pivot table e aggregazioni
- Time series e operazioni temporali
- Export e reportistica

### 1. Generazione dei Dataset di Base

Normalmente caricherebbe da file CSV reali.
Per questo esercizio, creiamo dati simulati.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === DATASET 1: CLIENTI ===
np.random.seed(42)
clienti = pd.DataFrame({
    'ClienteID': range(1, 51),  # 50 clienti
    'Nome': [f'Cliente_{i}' for i in range(1, 51)],
    'Città': np.random.choice(['Roma', 'Milano', 'Napoli', 'Torino', 'Venezia'], 50),
    'Regione': np.random.choice(['Lazio', 'Lombardia', 'Campania', 'Piemonte', 'Veneto'], 50),
    'Data_Iscrizione': pd.date_range(start='2024-01-01', periods=50, freq='D')
})

print("📊 CLIENTI")
display(clienti.head(10))

In [ ]:
# === DATASET 2: ORDINI ===
np.random.seed(42)
n_ordini = 300
ordini = pd.DataFrame({
    'OrdineID': range(1, n_ordini + 1),
    'ClienteID': np.random.choice(clienti['ClienteID'], n_ordini),
    'ProdottoID': np.random.choice(range(1, 21), n_ordini),  # 20 prodotti
    'Data': pd.date_range(start='2025-01-01', periods=n_ordini, freq='6H'),
    'Quantità': np.random.randint(1, 10, n_ordini),
    'Prezzo_Unitario': np.random.uniform(10, 500, n_ordini).round(2)
})

# Calcola l'importo totale
ordini['Importo'] = (ordini['Quantità'] * ordini['Prezzo_Unitario']).round(2)

print("📦 ORDINI")
display(ordini.head(10))

In [ ]:
# === DATASET 3: PRODOTTI ===
prodotti = pd.DataFrame({
    'ProdottoID': range(1, 21),
    'Nome_Prodotto': [
        'Laptop', 'Mouse', 'Monitor', 'Tastiera', 'Webcam',
        'Cuffie', 'Speaker', 'Hard Drive', 'RAM', 'CPU Cooler',
        'Power Supply', 'Case PC', 'Scheda Madre', 'SSD', 'GPU',
        'Router', 'Switch Rete', 'Cavo HDMI', 'Staffa Monitor', 'Hub USB'
    ],
    'Categoria': np.random.choice(['PC', 'Periferiche', 'Storage', 'Networking'], 20),
    'Margine_Profitto': np.random.uniform(0.15, 0.50, 20).round(2)  # Percentuale profitto
})

print("🛍️ PRODOTTI")
display(prodotti.head(10))

### 2. Merge dei Dataset

Combiniamo i tre dataset per avere una visione completa.

In [ ]:
# MERGE 1: Ordini + Clienti
vendite = pd.merge(ordini, clienti, on='ClienteID', how='left')

# MERGE 2: Risultato + Prodotti
vendite = pd.merge(vendite, prodotti, on='ProdottoID', how='left')

# Calcoliamo il profitto stimato
vendite['Profitto'] = (vendite['Importo'] * vendite['Margine_Profitto']).round(2)

print(f"✅ Dataset merged: {vendite.shape[0]} ordini totali")
display(vendite.head(10))

### 3. Filtri Avanzati

In [ ]:
# Filtro 1: Ordini ad alto valore (Importo > 2000€)
ordini_premium = vendite[vendite['Importo'] > 2000]
print(f"💰 Ordini Premium (>2000€): {len(ordini_premium)}")

# Filtro 2: Clienti da Milano OR Roma
clienti_nord_centro = vendite[vendite['Città'].isin(['Milano', 'Roma'])]
print(f"🏙️ Vendite Nord/Centro: {len(clienti_nord_centro)}")

# Filtro 3: Prodotti della categoria 'PC' con margine alto
pc_redditizi = vendite[
    (vendite['Categoria'] == 'PC') & 
    (vendite['Margine_Profitto'] > 0.35)
]
print(f"💻 PC Redditizi: {len(pc_redditizi)}")

# Filtro 4: Ordini recenti (ultimi 7 giorni)
data_limit = pd.to_datetime('2025-01-01') + pd.Timedelta(days=7)
ordini_recenti = vendite[vendite['Data'] <= data_limit]
print(f"📅 Ordini Recenti (7 giorni): {len(ordini_recenti)}")

### 4. Aggregazioni e Statistiche

In [ ]:
# Statistiche generali
print("=== STATISTICHE GENERALI ===")
print(f"Fatturato Totale: €{vendite['Importo'].sum():,.2f}")
print(f"Profitto Totale: €{vendite['Profitto'].sum():,.2f}")
print(f"Ordine Medio: €{vendite['Importo'].mean():,.2f}")
print(f"Ordine Massimo: €{vendite['Importo'].max():,.2f}")
print(f"Ordine Minimo: €{vendite['Importo'].min():,.2f}")

# Numero di clienti unici
print(f"\n👥 Clienti Unici: {vendite['ClienteID'].nunique()}")
print(f"📊 Numero Ordini: {len(vendite)}")

### 5. Analisi per Cliente

In [ ]:
# Raggruppamento per cliente
per_cliente = vendite.groupby('Nome').agg({
    'OrdineID': 'count',          # Numero ordini
    'Importo': ['sum', 'mean'],   # Total e media
    'Profitto': 'sum',             # Profitto totale
    'Città': 'first'               # Città (stessa per cliente)
}).round(2)

# Rinominiamo le colonne per chiarezza
per_cliente.columns = ['Num_Ordini', 'Fatturato_Totale', 'Ordine_Medio', 'Profitto_Totale', 'Città']

# Ordiniamo per fatturato decrescente
per_cliente = per_cliente.sort_values('Fatturato_Totale', ascending=False)

print("🏆 TOP 10 CLIENTI PER FATTURATO")
display(per_cliente.head(10))

### 6. Analisi per Categoria Prodotto

In [ ]:
# Pivot table per categoria
per_categoria = vendite.groupby('Categoria').agg({
    'OrdineID': 'count',
    'Importo': 'sum',
    'Quantità': 'sum',
    'Profitto': 'sum'
}).round(2)

per_categoria.columns = ['Num_Ordini', 'Fatturato', 'Unità_Vendute', 'Profitto']
per_categoria = per_categoria.sort_values('Fatturato', ascending=False)

print("📊 VENDITE PER CATEGORIA")
display(per_categoria)

### 7. Analisi Temporale

In [ ]:
# Estraiamo data e ora
vendite['Data_Solo'] = vendite['Data'].dt.date
vendite['Ora'] = vendite['Data'].dt.hour

# Vendite per giorno
per_giorno = vendite.groupby('Data_Solo')['Importo'].agg(['sum', 'count']).round(2)
per_giorno.columns = ['Fatturato_Giorno', 'Num_Ordini']

print("📅 PRIME 10 GIORNI")
display(per_giorno.head(10))

# Visualizzazione
per_giorno['Fatturato_Giorno'].plot(figsize=(12, 5), title='Fatturato Giornaliero')
plt.ylabel('Fatturato (€)')
plt.xlabel('Data')
plt.grid(True)
plt.show()

### 8. Analisi Geografica

In [ ]:
# Vendite per città
per_città = vendite.groupby('Città').agg({
    'Importo': 'sum',
    'OrdineID': 'count',
    'ClienteID': 'nunique'  # Clienti unici
}).round(2)

per_città.columns = ['Fatturato', 'Num_Ordini', 'Clienti_Unici']
per_città = per_città.sort_values('Fatturato', ascending=False)

print("🗺️ VENDITE PER CITTÀ")
display(per_città)

# Grafico
per_città['Fatturato'].plot(kind='bar', figsize=(10, 5), color='teal')
plt.title('Fatturato per Città')
plt.ylabel('Fatturato (€)')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

### 9. Pivot Table Avanzata

In [ ]:
# Pivot: Fatturato per Città e Categoria
pivot_città_cat = pd.pivot_table(
    vendite,
    values='Importo',
    index='Città',
    columns='Categoria',
    aggfunc='sum',
    fill_value=0
).round(2)

print("🎯 FATTURATO: CITTÀ vs CATEGORIA")
display(pivot_città_cat)

# Heatmap visuale
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_città_cat, annot=True, fmt='.0f', cmap='YlGn')
plt.title('Fatturato: Città vs Categoria')
plt.tight_layout()
plt.show()

### 10. Export Finali

In [ ]:
# Esporta i report principali
per_cliente.to_csv('report_clienti.csv')
per_categoria.to_csv('report_categorie.csv')
per_città.to_csv('report_città.csv')
pivot_città_cat.to_csv('report_pivot.csv')

# Esporta dati completi con filtri
ordini_premium.to_csv('ordini_premium.csv', index=False)

print("✅ Report esportati:")
print("  - report_clienti.csv")
print("  - report_categorie.csv")
print("  - report_città.csv")
print("  - report_pivot.csv")
print("  - ordini_premium.csv")